In [111]:
import pandas as pd
import numpy as np
import json


pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 20)

In [2]:
with open('../configs/app_config.json') as json_file:
    config = json.load(json_file)

In [3]:
config

{'canc_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/canc_df.csv',
 'promo_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/promo_df.csv',
 'sales_plan_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/sales_plan.csv',
 'sample_1000_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/sample_1000.csv',
 'transactions_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/transactions.csv',
 'wholesale_trade_table_path': '/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/wholesale_trade_table.csv'}

In [23]:
df_trans = pd.read_csv(config['transactions_path'])
df_samples = pd.read_csv(config['sample_1000_path'])
df_canc = pd.read_csv(config['canc_path'])
df_promo = pd.read_csv(config['promo_path'])
df_sales_plan = pd.read_csv(config['sales_plan_path'])
df_wholesale = pd.read_csv(config['wholesale_trade_table_path'])

df_feedback = pd.read_csv('/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/feedback.csv')
df_primer_batch = pd.read_csv('/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/primer_batch.csv')
df_primer_posilki = pd.read_csv('/home/juravlik/PycharmProjects/DynamicPricingSystem/data/raw/primer_posilki.csv')

# EDA

In [24]:
df_trans.head()

,dates,SKU,user,price
0,2018-11-09,16019,44796,1252.0
1,2018-11-09,16019,44510,1252.0
2,2018-11-09,16019,19074,1252.0
3,2018-11-09,16019,34472,1252.0
4,2018-11-10,16019,48407,1252.0


In [25]:
df_trans['dates'] = pd.to_datetime(df_trans['dates'])
df_trans['year'] = df_trans['dates'].dt.year
df_trans['month'] = df_trans['dates'].dt.month
df_trans['day_num'] = df_trans['dates'].dt.day
df_trans['week'] = df_trans['dates'].dt.week

/tmp/ipykernel_17616/1313961277.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_trans['week'] = df_trans['dates'].dt.week


In [26]:
df_trans

,dates,SKU,user,price,year,month,day_num,week
0,2018-11-09,16019,44796,1252.0,2018,11,9,45
1,2018-11-09,16019,44510,1252.0,2018,11,9,45
2,2018-11-09,16019,19074,1252.0,2018,11,9,45
3,2018-11-09,16019,34472,1252.0,2018,11,9,45
4,2018-11-10,16019,48407,1252.0,2018,11,10,45
...,...,...,...,...,...,...,...,...
1334255,2019-04-21,16663,20223,2240.0,2019,4,21,16
1334256,2019-04-21,16663,25555,2240.0,2019,4,21,16
1334257,2019-04-21,16663,47340,2240.0,2019,4,21,16
1334258,2019-04-21,16663,23762,2240.0,2019,4,21,16


In [66]:
df_num_purchases = (
    df_trans
    .groupby(['dates', 'SKU'])['user']
    .agg(num_purchases='count')
    .reset_index()
)

In [73]:
df_num_purchases['num_purchases_lag1'] = df_num_purchases.groupby('SKU')['num_purchases'].shift(1)

In [74]:
df_num_purchases[df_num_purchases.SKU == 67369]

,dates,SKU,num_purchases,num_purchases_lag1
0,2018-01-01,67369,7,NaN
1,2018-01-02,67369,7,7.0
4,2018-01-03,67369,8,7.0
10,2018-01-04,67369,7,8.0
19,2018-01-05,67369,7,7.0
...,...,...,...,...
315478,2019-11-26,67369,6,7.0
316272,2019-11-27,67369,6,6.0
317069,2019-11-28,67369,7,6.0
317868,2019-11-29,67369,7,7.0


In [60]:
df_num_purchases

,dates,SKU,num_purchases
0,2018-01-01,67369,7
1,2018-01-02,67369,7
2,2018-01-03,26629,5
3,2018-01-03,41262,6
4,2018-01-03,67369,8
...,...,...,...
318951,2019-11-30,98842,2
318952,2019-11-30,99266,5
318953,2019-11-30,99310,2
318954,2019-11-30,99974,2


In [39]:
df_trans.head()

,dates,SKU,user,price,year,month,day_num,week
0,2018-11-09,16019,44796,1252.0,2018,11,9,45
1,2018-11-09,16019,44510,1252.0,2018,11,9,45
2,2018-11-09,16019,19074,1252.0,2018,11,9,45
3,2018-11-09,16019,34472,1252.0,2018,11,9,45
4,2018-11-10,16019,48407,1252.0,2018,11,10,45


In [55]:
df_test = df_trans.groupby(['dates', 'SKU'])['price'].agg(diff_price='nunique').reset_index()
df_test

,dates,SKU,diff_price
0,2018-01-01,67369,1
1,2018-01-02,67369,1
2,2018-01-03,26629,1
3,2018-01-03,41262,1
4,2018-01-03,67369,1
...,...,...,...
318951,2019-11-30,98842,1
318952,2019-11-30,99266,1
318953,2019-11-30,99310,1
318954,2019-11-30,99974,1


In [12]:
df_trans.tail()

,dates,SKU,user,price
1334255,2019-04-21,16663,20223,2240.0
1334256,2019-04-21,16663,25555,2240.0
1334257,2019-04-21,16663,47340,2240.0
1334258,2019-04-21,16663,23762,2240.0
1334259,2019-04-21,16663,48007,2240.0


In [6]:
df_samples.head()

,sku_id,fincode,ui1_code,ui2_code,ui3_code,vendor,brand_code,creation_date,expiration_date
0,16663,Z8,Z81,Z8100,Z810000,FI3ASO,THSTSG81,2018-06-26,2019-04-21 00:00:00
1,91775,Q5,Q51,Q5100,Q510008,4O4WSH,WKXRWTP7,2019-07-07,2200-01-01 00:00:00
2,24641,ZR,ZR1,ZR100,ZR10000,71SM2O,J8XKU94W,2019-05-22,2200-01-01 00:00:00
3,84184,UA,UA1,UA102,UA10205,FI3ASO,5IEJMU4O,2018-04-19,2019-06-19 00:00:00
4,21070,XG,XG1,XG100,XG10000,4O4WSH,4WL34OA2,2019-06-27,2200-01-01 00:00:00


In [34]:
df_samples['creation_expiration_days'] = (pd.to_datetime(df_samples['expiration_date']) - pd.to_datetime(df_samples['creation_date'])).dt.days

In [35]:
df_samples

,sku_id,fincode,ui1_code,ui2_code,ui3_code,vendor,brand_code,creation_date,expiration_date,creation_expiration_days
0,16663,Z8,Z81,Z8100,Z810000,FI3ASO,THSTSG81,2018-06-26,2019-04-21 00:00:00,299
1,91775,Q5,Q51,Q5100,Q510008,4O4WSH,WKXRWTP7,2019-07-07,2200-01-01 00:00:00,65922
2,24641,ZR,ZR1,ZR100,ZR10000,71SM2O,J8XKU94W,2019-05-22,2200-01-01 00:00:00,65968
3,84184,UA,UA1,UA102,UA10205,FI3ASO,5IEJMU4O,2018-04-19,2019-06-19 00:00:00,426
4,21070,XG,XG1,XG100,XG10000,4O4WSH,4WL34OA2,2019-06-27,2200-01-01 00:00:00,65932
...,...,...,...,...,...,...,...,...,...,...
995,76988,UA,UA1,UA100,UA10000,UH2WWF,484JDFU5,2018-03-08,2019-08-10 00:00:00,520
996,30482,UA,UA1,UA100,UA10006,4HJSP5,8QJHUA65,2019-03-19,2200-01-01 00:00:00,66032
997,85915,BJ,BJ1,BJ102,BJ10201,AO59EQ,VMB2NPY2,2018-03-02,2019-07-05 00:00:00,490
998,39544,G9,G91,G9101,G910107,B4TA32,79VL731U,2019-05-11,2200-01-01 00:00:00,65979


In [56]:
df_canc

,year,week_num,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price
0,2019,1,10121,2945.681342,2965.743797,2917.394830
1,2019,1,10290,949.466060,942.301292,962.799009
2,2019,1,10973,3751.515022,3735.814981,3709.586540
3,2019,1,11179,3717.042306,3638.799082,3695.090121
4,2019,1,11680,1378.797669,1325.251312,1363.707489
...,...,...,...,...,...,...
10319,2019,52,97659,3409.219608,3394.456244,3434.336572
10320,2019,52,98346,1632.598644,1710.327024,1638.673052
10321,2019,52,98642,1498.441016,1520.235649,1481.583880
10322,2019,52,98774,1649.812737,1638.274072,1725.167364


In [8]:
df_promo.head()

,SKU,week_num,year,discount
0,74430,1,2018,0.15
1,69978,1,2018,0.05
2,81955,1,2018,0.15
3,78953,1,2018,0.10
4,43034,1,2018,0.05


In [42]:
df_promo.tail()

,SKU,week_num,year,discount
2337,35645,52,2019,0.15
2338,81314,52,2019,0.05
2339,75661,52,2019,0.05
2340,38664,52,2019,0.05
2341,47952,52,2019,0.10


In [9]:
df_sales_plan.head()

,SKU,year,month,plan,back_bonus
0,10290,2018,2,20.0,120000
1,10290,2018,3,60.0,140000
2,10290,2018,4,70.0,140000
3,10290,2018,5,60.0,140000
4,10290,2018,6,60.0,40000


In [10]:
df_wholesale.head()

,SKU,year,week_num,month,cost_price
0,10060,2019,1,12,2211.0
1,10060,2019,15,4,2454.0
2,10060,2019,16,4,2432.0
3,10060,2019,17,4,2449.0
4,10060,2019,18,4,2280.0


In [11]:
for col in df_samples.columns:
    print(col, '; nunique: ', df_samples[col].nunique())

sku_id ; nunique:  1000
fincode ; nunique:  30
ui1_code ; nunique:  31
ui2_code ; nunique:  91
ui3_code ; nunique:  377
vendor ; nunique:  24
brand_code ; nunique:  62
creation_date ; nunique:  522
expiration_date ; nunique:  162


# Check train set

In [106]:
df_train = pd.read_csv('/home/juravlik/PycharmProjects/DynamicPricingSystem/data/prepared/train_set.csv')

In [112]:
df_train

,dates,SKU,num_purchases,year,month,day,week_num,creation_date,expiration_date,creation_expiration_days,fincode_15,fincode_4W,fincode_6L,fincode_70,fincode_8J,fincode_AG,fincode_AQ,fincode_B0,fincode_B1,fincode_BJ,fincode_BY,fincode_EH,fincode_FE,fincode_G9,fincode_GV,fincode_GY,fincode_H7,fincode_K9,fincode_LT,fincode_M2,fincode_MB,fincode_Q5,fincode_RI,fincode_SI,fincode_UA,fincode_XG,fincode_YT,fincode_Z8,fincode_ZR,fincode_ZX,ui1_code_151,ui1_code_4W1,ui1_code_6L1,ui1_code_701,ui1_code_8J1,ui1_code_AG1,ui1_code_AQ1,ui1_code_B01,ui1_code_B11,ui1_code_BJ1,ui1_code_BY1,ui1_code_EH1,ui1_code_FE1,ui1_code_G91,ui1_code_GV1,ui1_code_GY1,ui1_code_H71,ui1_code_K91,ui1_code_LT1,ui1_code_M21,ui1_code_MB1,ui1_code_Q51,ui1_code_RI1,ui1_code_SI1,ui1_code_UA1,ui1_code_XG1,ui1_code_YT1,ui1_code_Z80,ui1_code_Z81,ui1_code_ZR1,ui1_code_ZX1,vendor_2CI2A8,vendor_3GFIMK,vendor_4HJSP5,vendor_4O4WSH,vendor_71SM2O,vendor_7I77VL,vendor_AO59EQ,vendor_AWD3XQ,vendor_B4TA32,vendor_BLW9FG,vendor_CHQKSO,vendor_FI3ASO,vendor_GVL335,vendor_H2OK0Z,vendor_JSOSD5,vendor_LDLJFP,vendor_SHL570,vendor_SJP384,vendor_T9BUA7,vendor_TJYT3A,vendor_UEYMBB,vendor_UH2WWF,vendor_VWHZMG,vendor_VZ44WY,brand_code_02Q78ADY,brand_code_1F5HYT5P,brand_code_1VPSWMTF,brand_code_20SL2XLU,brand_code_25T5WNRI,brand_code_28V0149U,brand_code_2XSAI69H,brand_code_38HT8FNL,brand_code_3FAIRJ4W,brand_code_3P2I0S6S,brand_code_484JDFU5,brand_code_4LU3GS87,brand_code_4P277OU8,brand_code_4WL34OA2,brand_code_52JETVXH,brand_code_5IEJMU4O,brand_code_5QQF7PWJ,brand_code_5XUYK688,brand_code_6VU0IOUP,brand_code_72PNE8FJ,brand_code_74CKJ9XQ,brand_code_79VL731U,brand_code_8FYENEWM,brand_code_8QJHUA65,brand_code_8TMVJREA,brand_code_9J3MMTSQ,brand_code_9XGMSC3X,brand_code_AXB8H17D,brand_code_F87SIC0N,brand_code_FJMH9IG1,brand_code_G6OB30M4,brand_code_GS4NPH2K,brand_code_H01VAI1S,brand_code_H0PQ4U24,brand_code_H1BWFJRN,brand_code_HDHOCYPD,brand_code_J8XKU94W,brand_code_JK4UKBI2,brand_code_JXS1Z5D7,brand_code_KU7Q9EM1,brand_code_LNOTDGF7,brand_code_M2LFH9WE,brand_code_MMC2P4Y3,brand_code_N2CAIPJF,brand_code_O0Q22TYD,brand_code_O8WJHL4Q,brand_code_Q6NGRHUA,brand_code_R36KIEMV,brand_code_SKXXW8V9,brand_code_THSTSG81,brand_code_UEKGPFHF,brand_code_UJWSTGEH,brand_code_UWKPEFCJ,brand_code_VCZNKWIS,brand_code_VMB2NPY2,brand_code_W581KQD3,brand_code_WKXRWTP7,brand_code_XF98E5PI,brand_code_YTE7JS4H,brand_code_YZ7ES5HU,brand_code_Z4ZXJ8UY,brand_code_Z7FUX49S,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus,cost_price,price,mean_canc_price,min_canc_price,max_canc_price,price_minus_min_canc_price,price_minus_max_canc_price,price_minus_mean_canc_price,price_div_min_canc_price,price_div_max_canc_price,price_div_mean_canc_price,promo_price,date_creation_date,expiration_dates_date,price_lag1,num_purchases_lag1,price_lag2,num_purchases_lag2,price_lag3,num_purchases_lag3,price_lag4,num_purchases_lag4,price_lag5,num_purchases_lag5,price_lag6,num_purchases_lag6,price_lag7,num_purchases_lag7
0,2018-01-01,67369,7,2018,1,1,1,2018-01-01,2200-01-01 00:00:00,66474,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,210.0,120000,3083.0,3607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,66474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,67369,7,2018,1,2,1,2018-01-01,2200-01-01 00:00:00,66474,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,210.0,120000,3083.0,3607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,66473,3607.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-

In [ ]:
df_train

In [113]:
drop_features = ['dates', 'SKU', 'creation_date', 'expiration_date']

In [114]:
df_train.drop(columns=drop_features)

,num_purchases,year,month,day,week_num,creation_expiration_days,fincode_15,fincode_4W,fincode_6L,fincode_70,fincode_8J,fincode_AG,fincode_AQ,fincode_B0,fincode_B1,fincode_BJ,fincode_BY,fincode_EH,fincode_FE,fincode_G9,fincode_GV,fincode_GY,fincode_H7,fincode_K9,fincode_LT,fincode_M2,fincode_MB,fincode_Q5,fincode_RI,fincode_SI,fincode_UA,fincode_XG,fincode_YT,fincode_Z8,fincode_ZR,fincode_ZX,ui1_code_151,ui1_code_4W1,ui1_code_6L1,ui1_code_701,ui1_code_8J1,ui1_code_AG1,ui1_code_AQ1,ui1_code_B01,ui1_code_B11,ui1_code_BJ1,ui1_code_BY1,ui1_code_EH1,ui1_code_FE1,ui1_code_G91,ui1_code_GV1,ui1_code_GY1,ui1_code_H71,ui1_code_K91,ui1_code_LT1,ui1_code_M21,ui1_code_MB1,ui1_code_Q51,ui1_code_RI1,ui1_code_SI1,ui1_code_UA1,ui1_code_XG1,ui1_code_YT1,ui1_code_Z80,ui1_code_Z81,ui1_code_ZR1,ui1_code_ZX1,vendor_2CI2A8,vendor_3GFIMK,vendor_4HJSP5,vendor_4O4WSH,vendor_71SM2O,vendor_7I77VL,vendor_AO59EQ,vendor_AWD3XQ,vendor_B4TA32,vendor_BLW9FG,vendor_CHQKSO,vendor_FI3ASO,vendor_GVL335,vendor_H2OK0Z,vendor_JSOSD5,vendor_LDLJFP,vendor_SHL570,vendor_SJP384,vendor_T9BUA7,vendor_TJYT3A,vendor_UEYMBB,vendor_UH2WWF,vendor_VWHZMG,vendor_VZ44WY,brand_code_02Q78ADY,brand_code_1F5HYT5P,brand_code_1VPSWMTF,brand_code_20SL2XLU,brand_code_25T5WNRI,brand_code_28V0149U,brand_code_2XSAI69H,brand_code_38HT8FNL,brand_code_3FAIRJ4W,brand_code_3P2I0S6S,brand_code_484JDFU5,brand_code_4LU3GS87,brand_code_4P277OU8,brand_code_4WL34OA2,brand_code_52JETVXH,brand_code_5IEJMU4O,brand_code_5QQF7PWJ,brand_code_5XUYK688,brand_code_6VU0IOUP,brand_code_72PNE8FJ,brand_code_74CKJ9XQ,brand_code_79VL731U,brand_code_8FYENEWM,brand_code_8QJHUA65,brand_code_8TMVJREA,brand_code_9J3MMTSQ,brand_code_9XGMSC3X,brand_code_AXB8H17D,brand_code_F87SIC0N,brand_code_FJMH9IG1,brand_code_G6OB30M4,brand_code_GS4NPH2K,brand_code_H01VAI1S,brand_code_H0PQ4U24,brand_code_H1BWFJRN,brand_code_HDHOCYPD,brand_code_J8XKU94W,brand_code_JK4UKBI2,brand_code_JXS1Z5D7,brand_code_KU7Q9EM1,brand_code_LNOTDGF7,brand_code_M2LFH9WE,brand_code_MMC2P4Y3,brand_code_N2CAIPJF,brand_code_O0Q22TYD,brand_code_O8WJHL4Q,brand_code_Q6NGRHUA,brand_code_R36KIEMV,brand_code_SKXXW8V9,brand_code_THSTSG81,brand_code_UEKGPFHF,brand_code_UJWSTGEH,brand_code_UWKPEFCJ,brand_code_VCZNKWIS,brand_code_VMB2NPY2,brand_code_W581KQD3,brand_code_WKXRWTP7,brand_code_XF98E5PI,brand_code_YTE7JS4H,brand_code_YZ7ES5HU,brand_code_Z4ZXJ8UY,brand_code_Z7FUX49S,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus,cost_price,price,mean_canc_price,min_canc_price,max_canc_price,price_minus_min_canc_price,price_minus_max_canc_price,price_minus_mean_canc_price,price_div_min_canc_price,price_div_max_canc_price,price_div_mean_canc_price,promo_price,date_creation_date,expiration_dates_date,price_lag1,num_purchases_lag1,price_lag2,num_purchases_lag2,price_lag3,num_purchases_lag3,price_lag4,num_purchases_lag4,price_lag5,num_purchases_lag5,price_lag6,num_purchases_lag6,price_lag7,num_purchases_lag7
0,7,2018,1,1,1,66474,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,210.0,120000,3083.0,3607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,66474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,2018,1,2,1,66474,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,210.0,120000,3083.0,3607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,66473,3607.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,2018,1,3,1,98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0